In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import re
from datetime import datetime, time

In [2]:
#df_test = pd.read_csv('C:/Users/kosty/SF DS Project 5/test.csv')

In [3]:
#df_test.iloc[0]

In [4]:
#df_test.iloc[20]['super_gen']

In [5]:
names = [
    'url',
    'complectation_link',
    'body',
    'brand',
    'color',
    'complectation',
    'description',
    'V_engine',
    'power',
    'fuel',
    'mileage',
    'model_date',
    'model',
    'doors',
    'parsing_time',
    'date',
    'transmition',
    'vendor',
    'owners',
    'ownership_time',
    'pts',
    'drive_type',
    'driving_wheel',
    'condition',
    'custom',
    'clearance',
    'V_trunk',
    'lenght',
    'width',
    'height',
    'full_weight',
    'curb_weight',
    'geers',
    'V_tank',
    'acceleration',
    'fuel_consumption',
    'max_speed',
    'seets',
    'turbo',
    'cylinders',
    'valves',
    'price',
    'iteration'
]

In [15]:
for name in names:
    exec('{}=[]'.format(name))

links_cars = open('C:/Users/kosty/SF DS Project 5/links_cars_save_1.txt')
i = -1
for link in links_cars:
    i = i+1
    #print(i)
    #while i < 100:
    #    i = i+1
    #    continue
    if i % 500 == 0:
        print(i)
    url_ = link[:-2]
    
    conection = '-'
    while conection == '-':
        try:
            page = requests.get(url_, headers={'User-Agent': 'Mozilla/5.0'})
        except OSError:
            conection = '-'
        else:
            conection = '+'    

    #page = requests.get(url_, headers={'User-Agent': 'Mozilla/5.0'})
    page.encoding = 'utf8'
    page5 = BeautifulSoup(page.text, 'html.parser')
    if len( page5.find_all('a', {'class': 'Link SpoilerLink CardCatalogLink SpoilerLink_type_default'})) == 0:
        continue
    complectation_link_ = page5.find_all(
        'a', {'class': 'Link SpoilerLink CardCatalogLink SpoilerLink_type_default'})[0].get('href')
   
    conection = '-'
    while conection == '-':
        try:
            page = requests.get(complectation_link_, headers={'User-Agent': 'Mozilla/5.0'})
        except OSError:
            conection = '-'
        else:
            conection = '+'        
    
    #page = requests.get(complectation_link_, headers={'User-Agent': 'Mozilla/5.0'})
    page.encoding = 'utf8'
    page6 = BeautifulSoup(page.text, 'html.parser')
    url.append(url_)
    complectation_link.append(complectation_link_)
    name_df = []
    value_df = []
    l = 0
    class_ = page6.find_all('dl')
    for dl in class_:
        for child_dl in dl.children:
            if l%2 ==0:
                name_df.append(child_dl.contents[0])
            else:
                value_df.append(child_dl.contents[0])
            l = l+1
    car_info = pd.DataFrame()
    car_info['name'] = pd.Series(name_df)
    car_info['value'] = pd.Series(value_df)
# Тип кузова
    body_type = page6.find_all(
        'a', {'class': 'link link_pseudo search-form-v2-mmm__breadcrumbs-item search-form-v2-mmm__breadcrumbs-item_state_selected search-form-v2-mmm__breadcrumbs-item_type_configuration link__control i-bem'})
    if len(body_type) == 0:
        body_type = ''
    else:
        body_type = body_type[0].contents[0]
    body.append(body_type)
# Брэнд
    brand_ = page6.find_all(
        'a', {'class': 'link link_pseudo search-form-v2-mmm__breadcrumbs-item search-form-v2-mmm__breadcrumbs-item_state_selected search-form-v2-mmm__breadcrumbs-item_type_mark link__control i-bem'})
    if len(brand_) == 0:
        brand_ = ''
    else:
        brand_ = brand_[0].contents[0]
    brand.append(brand_)
# Цвет
    color_ = page5.find_all('a', {'class' : 'Link Link_color_black'})
    if len(color_) == 0:
        color_ = page5.find_all('a', {'class' :'Link CardInfoGrouped__cellValue'})
        if len(color_) < 3:
            color_ = color_[-1].contents[0] 
        else:
            color_ = color_[2].contents[0]
    else:
        color_ = color_[2].contents[0]
    color.append(color_)
# complectation_dict
    complectation_dict = page5.find_all('h2', {'class' :'CardComplectation__title'})
    if len(complectation_dict) == 0:
        complectation_dict = ''
    else:
        complectation_dict = complectation_dict[0].contents[0]
    complectation.append(complectation_dict)
# description
    description_ = page5.find_all('div', {'class' : 'CardDescription__textInner'})
    if len(description_) == 0:
        description_ = ''
    else:
        description_ = page5.find_all('div', {'class' : 'CardDescription__textInner'})[0].find_all('span')[0].contents
    description.append(description_)
# engineDisplacement Объем двигателя, см³
    engine_displacement = car_info['value'][car_info['name'] == 'Объем двигателя, см³']
    if len(engine_displacement) == 0:
        V_engine.append('')
    else:
        V_engine.append(engine_displacement.values[0])
# engine power Максимальная мощность, л.с./кВт при об/мин
    engine_power = car_info['value'][car_info['name'] == 'Максимальная мощность, л.с./кВт при об/мин']
    if len(engine_power) == 0:
        power.append('')
    else:
        power.append(engine_power.values[0])
# fuel_type Тип двигателя
    fuel_type = car_info['value'][car_info['name'] == 'Тип двигателя']
    if len(fuel_type) == 0:
        fuel.append('')
    else:    
        fuel.append(fuel_type.values[0])
# mileage
    mileage_ = page5.find_all('span', {'class' : 'CardInfoRow__cell'})
    if len(mileage_)<4:
        mileage_ = 0
    else:
        mileage_ = mileage_[3].contents
        mileage_ = re.findall('\d+', str(mileage_))
        if len(mileage_) == 1:
            mileage_ = float(mileage_[0])/100
        elif len(mileage_) == 2:
            mileage_ = float(mileage_[0] + mileage_[1])/100
        elif len(mileage_) == 3:
            mileage_ = float(mileage_[0] + mileage_[1] + mileage_[2])/100
        elif len(mileage_) == 4:
            mileage_ = float(mileage_[0] + mileage_[1] + mileage_[2] + mileage_[3])/100        
    mileage.append(mileage_)
# model_date
    modeldate = page6.find_all('a', {'class' : 'link link_pseudo search-form-v2-mmm__breadcrumbs-item search-form-v2-mmm__breadcrumbs-item_state_selected search-form-v2-mmm__breadcrumbs-item_type_generation link__control i-bem'})
    if len(modeldate) == 0:
        modeldate = ''
    else:
        modeldate = re.findall('\d{4}', str(modeldate[0].contents))
        modeldate = int(modeldate[0])
    model_date.append(modeldate)
# model_name
    model_name = page6.find_all('a', {'class' : 'link link_pseudo search-form-v2-mmm__breadcrumbs-item search-form-v2-mmm__breadcrumbs-item_state_selected search-form-v2-mmm__breadcrumbs-item_type_model link__control i-bem'})
    if len(model_name) == 0:
        model_name = ''
    else:
        model_name = model_name[0].contents[0]
    model.append(model_name)
# number_of_doors Количество дверей
    number_of_doors = car_info['value'][car_info['name'] == 'Количество дверей']
    if len(number_of_doors) == 0:
        doors.append('')
    else:
        doors.append(number_of_doors.values[0])
# parcing_unixtime
    parsing_unixtime = datetime.now().timestamp()
    parsing_time.append(parsing_unixtime)
# productionDate
    productionDate = page5.find_all('a', {'Link Link_color_black'})
    if len(productionDate) == 0:
        productionDate = 2021
    else:
        productionDate = productionDate[0].contents[0]
    date.append(productionDate)
# transmition Коробка передач
    transmition_ = car_info['value'][car_info['name'] == 'Коробка передач']
    if len(transmition_) == 0:
        transmition.append('')
    else:    
        transmition.append(transmition_.values[0])
# vendor Страна марки
    vendor_ = car_info['value'][car_info['name'] == 'Страна марки']
    if len(vendor_) == 0:
        vendor.append('')
    else:       
        vendor.append(vendor_.values[0])
# owners
    owners_ = page5.find_all('li', {'class' : 'CardInfoRow CardInfoRow_ownersCount'})
    if len(owners_) ==0:
        owners_ = 0
    else:
        owners_ = page5.find_all('li', {'class' : 'CardInfoRow CardInfoRow_ownersCount'})[0]
        owners_ = owners_.find_all('span', {'class' :'CardInfoRow__cell'})
        owners_ = re.findall('\d+', str(owners_[1]))[0]
    owners.append(owners_)
# own_time
    own_time = page5.find_all('li', {'class' : 'CardInfoRow CardInfoRow_owningTime'})
    if len(own_time) == 0:
        own_time = ''
    else:
        own_time = page5.find_all('li', {'class' : 'CardInfoRow CardInfoRow_owningTime'})[0]
        own_time = own_time.find_all('span', {'class' :'CardInfoRow__cell'})
        own_time = own_time[1].contents[0]
    ownership_time.append(own_time)
# PTS
    pts_ = page5.find_all('li', {'class' : 'CardInfoRow CardInfoRow_pts'})
    if len(pts_) == 0:
        pts_ = ''
    else:
        pts_ = page5.find_all('li', {'class' : 'CardInfoRow CardInfoRow_pts'})[0]
        pts_ = pts_.find_all('span', {'class' :'CardInfoRow__cell'})
        pts_ = pts_[1].contents[0]
    
    pts.append(pts_)
# driving_wheels Тип привода
    dw_ = car_info['value'][car_info['name'] == 'Тип привода']
    if len(dw_) == 0:
        drive_type.append('')
    else:         
        drive_type.append(dw_.values[0])
# steering_wheel
    steering_wheel = page5.find_all('li', {'class' : 'CardInfoRow CardInfoRow_wheel'})
    if len(re.findall('правый', str(steering_wheel).lower())) > 0:
        steering_wheel = 'правый'
    else:
        steering_wheel = 'левый'
    driving_wheel.append(steering_wheel)
# condition
    condition_ = page5.find_all('li', {'class' : 'CardInfoRow CardInfoRow_state'})
    if len(condition_) > 0:
        condition_ = condition_[0].contents
        condition_ = condition_[1].contents[0]
    else:
        condition_ = ''
    condition.append(condition_)
# customs    
    customs = page5.find_all('li', {'class' : 'CardInfoRow CardInfoRow_customs'})
    if len(customs) > 0:
        customs = customs[0].contents
        customs = customs[1].contents[0]
    else:
        customs = ''    
    custom.append(customs)
# clearance Клиренс
    clearance_ = car_info['value'][car_info['name'] == 'Клиренс']
    if len(clearance_) == 0:
        clearance.append('')
    else:  
        clearance.append(clearance_.values[0])
# V_trunk Объем багажника мин/макс, л
    V_trunk_ = car_info['value'][car_info['name'] == 'Объем багажника мин/макс, л']
    if len(V_trunk_) == 0:
        V_trunk.append('')
    else:
        V_trunk.append(V_trunk_.values[0])
# lenght Длина
    lenght_ = car_info['value'][car_info['name'] == 'Длина']
    if len(lenght_) == 0:
        lenght.append('')
    else:    
        lenght.append(lenght_.values[0])
# width Ширина
    width_ = car_info['value'][car_info['name'] == 'Ширина']
    if len(width_) == 0:
        width.append('')
    else: 
        width.append(width_.values[0])
# height Высота
    height_ = car_info['value'][car_info['name'] == 'Высота']
    if len(height_) == 0:
        height.append('')
    else: 
        height.append(height_.values[0])
# full_weight Полная масса, кг
    full_weight_ = car_info['value'][car_info['name'] == 'Полная масса, кг']
    if len(full_weight_) == 0:
        full_weight.append('')
    else:
        full_weight.append(full_weight_.values[0])
# curb_weight Снаряженная масса, кг
    curb_weight_ = car_info['value'][car_info['name'] == 'Снаряженная масса, кг']
    if len(curb_weight_) == 0:
        curb_weight.append('')
    else: 
        curb_weight.append(curb_weight_.values[0])
# geers Количество передач
    geers_ = car_info['value'][car_info['name'] == 'Количество передач']
    if len(geers_) == 0:
        geers.append('')
    else: 
        geers.append(geers_.values[0])
# V_tank Объём топливного бака, л
    V_tank_ = car_info['value'][car_info['name'] == 'Объём топливного бака, л']
    if len(V_tank_) == 0:
        V_tank.append('')
    else: 
        V_tank.append(V_tank_.values[0])
# acceleration  Разгон до 100 км/ч, с
    acceleration_ = car_info['value'][car_info['name'] == 'Разгон до 100 км/ч, с']
    if len(acceleration_) == 0:
        acceleration.append('')
    else:
        acceleration.append(acceleration_.values[0])
# fuel_consumption Расход топлива, л смешанный
    fuel_consumption_ = car_info['value'][car_info['name'] == 'Расход топлива, л смешанный']
    if len(fuel_consumption_) == 0:
        fuel_consumption.append('')
    else:
        fuel_consumption.append(fuel_consumption_.values[0])
# max_speed Максимальная скорость, км/ч
    max_speed_ = car_info['value'][car_info['name'] == 'Максимальная скорость, км/ч']
    if len(max_speed_) == 0:
        max_speed.append('')
    else:
        max_speed.append(max_speed_.values[0])
# seets Количество мест
    seets_ = car_info['value'][car_info['name'] == 'Количество мест']
    if len(seets_) == 0:
        seets.append('')
    else:
        seets.append(seets_.values[0])
# turbo Тип наддува
    turbo_ = car_info['value'][car_info['name'] == 'Тип наддува']
    if len(turbo_) == 0:
        turbo.append('')
    else:
        turbo.append(turbo_.values[0])
# cylinders Количество цилиндров
    cylinders_ = car_info['value'][car_info['name'] == 'Количество цилиндров']
    if len(cylinders_) == 0:
        cylinders.append('')
    else:
        cylinders.append(cylinders_.values[0])
# valves Число клапанов на цилиндр
    valves_ = car_info['value'][car_info['name'] == 'Число клапанов на цилиндр']
    if len(valves_) == 0:
        valves.append('')
    else:
        valves.append(valves_.values[0])
# price
    price_ = page5.find_all('span', {'OfferPriceCaption__price'})
    if len(price_) == 0:
        price_ = ''
    else:
        price_ = re.findall('\d+',str(price_[0]))
        price_1 = ''
        for part in price_:
            price_1 = price_1 + part
        price_ = int(price_1)
    price.append(price_)
# iteration
    iteration.append(i)
    if i%1000 == 0 and i != 0:
        print('Я обработал {} строк, и сохранил очередной csv'.format(i))
        df_car = pd.DataFrame()
        for name in names:
            df_car[name] = eval(name)
        df_car.to_csv('C:/Users/kosty/SF DS Project 5/df_cars 1/df_cars_{}.csv'.format(i), sep = ';')
        df_car = pd.DataFrame()
        for name in names:
            exec('{}=[]'.format(name))
    #if i == 3:
    #    break

0
500


C:\Users\kosty\programms\anaconda3\lib\site-packages\ipykernel_launcher.py:59: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
C:\Users\kosty\programms\anaconda3\lib\site-packages\ipykernel_launcher.py:60: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
C:\Users\kosty\programms\anaconda3\lib\site-packages\pandas\core\ops\array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


1000
Я обработал 1000 строк, и сохранил очередной csv
1500
2000
Я обработал 2000 строк, и сохранил очередной csv
2500
3000
Я обработал 3000 строк, и сохранил очередной csv
3500
4000
Я обработал 4000 строк, и сохранил очередной csv
4500
5000
Я обработал 5000 строк, и сохранил очередной csv
5500
6000
Я обработал 6000 строк, и сохранил очередной csv
6500
7000
Я обработал 7000 строк, и сохранил очередной csv
7500
8000
Я обработал 8000 строк, и сохранил очередной csv
8500
9000
Я обработал 9000 строк, и сохранил очередной csv
9500
10000
Я обработал 10000 строк, и сохранил очередной csv
10500
11000
Я обработал 11000 строк, и сохранил очередной csv
11500
12000
Я обработал 12000 строк, и сохранил очередной csv
12500
13000
Я обработал 13000 строк, и сохранил очередной csv
13500
14000
Я обработал 14000 строк, и сохранил очередной csv
14500
15000
Я обработал 15000 строк, и сохранил очередной csv
15500
16000
Я обработал 16000 строк, и сохранил очередной csv
16500
17000
Я обработал 17000 строк, и со

132500
133000
Я обработал 133000 строк, и сохранил очередной csv
133500
134000
Я обработал 134000 строк, и сохранил очередной csv
134500
135000
Я обработал 135000 строк, и сохранил очередной csv
135500
136000
Я обработал 136000 строк, и сохранил очередной csv
136500
137000
Я обработал 137000 строк, и сохранил очередной csv
137500
138000
Я обработал 138000 строк, и сохранил очередной csv
138500
139000
Я обработал 139000 строк, и сохранил очередной csv
139500
140000
Я обработал 140000 строк, и сохранил очередной csv
140500
141000
Я обработал 141000 строк, и сохранил очередной csv
141500
142000
Я обработал 142000 строк, и сохранил очередной csv
142500
143000
Я обработал 143000 строк, и сохранил очередной csv
143500
144000
Я обработал 144000 строк, и сохранил очередной csv
144500
145000
Я обработал 145000 строк, и сохранил очередной csv
145500
146000
Я обработал 146000 строк, и сохранил очередной csv
146500
147000
Я обработал 147000 строк, и сохранил очередной csv
147500
148000
Я обработал